# Encoding

In [69]:
import os
import re
import sys
import warnings

import copy
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

warnings.filterwarnings('ignore')

In [70]:
sys.path.append(os.path.join('..', 'src'))

# define functions

In [71]:
import importlib
import utils
importlib.reload(utils)
from utils import build_data_dict

In [72]:
def ordinal_encode(df, df_test, testing = False):
#     df = df.copy()
    categories_dict = {}
    
    temp_merge = df.append(df_test)
    for cat in temp_merge.columns:
        if temp_merge[cat].dtypes == 'object':
            categories_dict[cat] = list(temp_merge[cat].unique())
            if testing:
                print("Numero de categorias para variavel '{}': {} ".format(cat,temp_merge[cat].unique().size))

    if testing:
        print()
        print(list(categories_dict.keys()))
        
    enc = OrdinalEncoder(categories=list(categories_dict.values()))
    trained_encoder = enc.fit(df[list(categories_dict.keys())])
    
    # transform train and test
    df[list(categories_dict.keys())] = trained_encoder.transform(df[list(categories_dict.keys())])
    df_test[list(categories_dict.keys())] = trained_encoder.transform(df_test[list(categories_dict.keys())])

    if testing:
        print(categories_dict)
    
    return df, df_test

def one_hot_encode(df):
    print('Quantity of columns before one-hot encoding:', len(df.columns))
    
    df_oldcols = df.columns.to_list()
    df = pd.get_dummies(df, prefix_sep='_', drop_first=True)
    
    print('Quantity of columns after one-hot encoding:', len(df.columns))
    
    # rename columns to show which are dummies
    onehot_cols = list(set(df.columns.to_list()) - set(df_oldcols))
    onehot_cols_renaming = {col: 'dummy_'+col.replace('-', '_') for col in onehot_cols}
    df.rename(columns = onehot_cols_renaming, inplace=True)
    
    return df

# Define paths and capture data

In [73]:
inputs = os.path.join('..', 'data', '02_intermediate')
outputs = os.path.join('..', 'data', '03_processed')
reports = os.path.join('..', 'data', '06_reporting')

data_list = ['X_train', 'X_test']
ord_dict = build_data_dict(inputs, data_list)
oh_dict = copy.deepcopy(ord_dict)

y_dict = build_data_dict(inputs, ['y_train', 'y_test'])

loading data into dictionary
loading data into dictionary


In [74]:
report_df = pd.read_csv(os.path.join(reports, 'data_types.csv'))

# Count categorical data unique values
Check both train and test. Any inconsistency between them should be addressed.

In [75]:
for data in ['X_train', 'X_test']:
    categories_dict = {}
    print('\r\nchecking number of categories for {}'. format(data))
    for cat in ord_dict[data].columns:
        if ord_dict[data][cat].dtypes == 'object':
            categories_dict[cat] = list(ord_dict[data][cat].unique())
            print("Numero de categorias para variavel '{}': {} ".format(cat, ord_dict[data][cat].unique().size))


checking number of categories for X_train
Numero de categorias para variavel 'workclass': 7 
Numero de categorias para variavel 'education': 16 
Numero de categorias para variavel 'marital_status': 7 
Numero de categorias para variavel 'occupation': 14 
Numero de categorias para variavel 'relationship': 6 
Numero de categorias para variavel 'race': 5 
Numero de categorias para variavel 'sex': 2 
Numero de categorias para variavel 'native_country': 39 

checking number of categories for X_test
Numero de categorias para variavel 'workclass': 8 
Numero de categorias para variavel 'education': 16 
Numero de categorias para variavel 'marital_status': 7 
Numero de categorias para variavel 'occupation': 14 
Numero de categorias para variavel 'relationship': 6 
Numero de categorias para variavel 'race': 5 
Numero de categorias para variavel 'sex': 2 
Numero de categorias para variavel 'native_country': 40 


# Ordinal Encoding

In [76]:
ord_dict['X_train'], ord_dict['X_test'] = ordinal_encode(ord_dict['X_train'], ord_dict['X_test'], testing = False)

ord_dict['X_train'].head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country
id,,,,,,,,,,,,,
14160,27.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.0,0.0
27048,45.0,1.0,1.0,9.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,40.0,0.0
28868,29.0,0.0,2.0,13.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,55.0,0.0
5667,30.0,0.0,2.0,13.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0
7827,29.0,2.0,0.0,10.0,0.0,3.0,0.0,0.0,1.0,2202.0,0.0,50.0,0.0


# One-Hot Encoding

In [77]:
for df in ['X_train', 'X_test']:
    oh_dict[df] = one_hot_encode(oh_dict[df])
    
print('\r\nColumns of the new database:')
print(oh_dict[df].columns.to_list())

Quantity of columns before one-hot encoding: 13
Quantity of columns after one-hot encoding: 93
Quantity of columns before one-hot encoding: 13
Quantity of columns after one-hot encoding: 95

Columns of the new database:
['age', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week', 'dummy_workclass_Local_gov', 'dummy_workclass_Never_worked', 'dummy_workclass_Private', 'dummy_workclass_Self_emp_inc', 'dummy_workclass_Self_emp_not_inc', 'dummy_workclass_State_gov', 'dummy_workclass_Without_pay', 'dummy_education_11th', 'dummy_education_12th', 'dummy_education_1st_4th', 'dummy_education_5th_6th', 'dummy_education_7th_8th', 'dummy_education_9th', 'dummy_education_Assoc_acdm', 'dummy_education_Assoc_voc', 'dummy_education_Bachelors', 'dummy_education_Doctorate', 'dummy_education_HS_grad', 'dummy_education_Masters', 'dummy_education_Preschool', 'dummy_education_Prof_school', 'dummy_education_Some_college', 'dummy_marital_status_Married_AF_spouse', 'dummy_marital_status_Married_ci

In [78]:
oh_dict['X_test']

,age,education_num,capital_gain,capital_loss,hours_per_week,dummy_workclass_Local_gov,dummy_workclass_Never_worked,dummy_workclass_Private,dummy_workclass_Self_emp_inc,dummy_workclass_Self_emp_not_inc,...,dummy_native_country_Portugal,dummy_native_country_Puerto_Rico,dummy_native_country_Scotland,dummy_native_country_South,dummy_native_country_Taiwan,dummy_native_country_Thailand,dummy_native_country_Trinadad&Tobago,dummy_native_country_United_States,dummy_native_country_Vietnam,dummy_native_country_Yugoslavia
id,,,,,,,,,,,,,,,,,,,,,
0,25.0,7.0,0.0,0.0,40.0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38.0,9.0,0.0,0.0,50.0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,28.0,12.0,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,44.0,10.0,7688.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,18.0,10.0,0.0,0.0,30.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,39.0,13.0,0.0,0.0,36.0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
16277,64.0,9.0,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
16278,38.0,13.0,0.0,0.0,50.0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


# report new data types

In [79]:
dummy_vars_df = pd.DataFrame([c for c in oh_dict['X_train'].columns.to_list() if c.startswith('dummy')])
report_df = pd.concat([report_df,dummy_vars_df], ignore_index=True, axis=1)
report_df.columns = ['numerical_cols', 'non_numerical_cols', 'dummy_cols']

### data alignment
if some category is missing on test set, we need to account for that and build corresponding column filled with 'zeros'.

In [80]:
def fill_missing_cols(smaller, greater):
    missing_cols = set( greater.columns ) - set( smaller.columns )
    for c in missing_cols:
        smaller[c] = 0
    
    return smaller

In [81]:
oh_dict['X_train'] = fill_missing_cols(oh_dict['X_train'], oh_dict['X_test'])
oh_dict['X_test'] = fill_missing_cols(oh_dict['X_test'], oh_dict['X_train'])

# align column positions (no data leakage here. Just altering column ordering.)
oh_dict['X_train'], oh_dict['X_test'] = oh_dict['X_train'].align(oh_dict['X_test'], axis=1)

# Save processed data

In [84]:
for df in ['X_train', 'X_test']:
    ord_dict[df].to_csv(os.path.join(outputs, df+'.csv'))
    oh_dict[df].to_csv(os.path.join(outputs, df+'_oh.csv'))
    
for df in ['X_train', 'X_test']:
    print(ord_dict[df].shape)
    print(oh_dict[df].shape)
    
for df in ['y_train', 'y_test']:
    y_dict[df].to_csv(os.path.join(outputs, df+'.csv'))

(4995, 13)
(4995, 95)
(16281, 13)
(16281, 95)


# save report over data types

In [85]:
report_df.to_csv(os.path.join(reports, 'data_types.csv'), index=False)